In [1]:
import ipsuite as ips

2023-07-26 16:06:23,675 (DEBUG): Welcome to IPS - the Interatomic Potential Suite!


DFT based on https://www.sciencedirect.com/science/article/pii/S1386142521004455#b0030

In [3]:
thermostat = ips.calculators.LangevinThermostat(
    temperature=298.15, friction=0.01, time_step=0.5
)

uncertainty_check = ips.analysis.ThresholdCheck(
    value="energy_uncertainty", max_value=2.0, larger_only=True
)

mapping = ips.geometry.BarycenterMapping(data=None)

with ips.Project(automatic_node_names=True) as project:
    mol = ips.configuration_generation.SmilesToAtoms(smiles="CS(=O)C")

    # Create a box of atoms.
    packmol = ips.configuration_generation.Packmol(
        data=[mol.atoms], count=[38], density=1095.2
    )

    # Define the CP2K calculations
    cp2k = ips.calculators.CP2KSinglePoint(
        data=packmol.atoms,
        cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
        cp2k_shell="cp2k_shell.ssmp",
    )

    geopt = ips.calculators.ASEGeoOpt(
        model=cp2k,
        data=packmol.atoms,
        optimizer="BFGS",
        run_kwargs={"fmax": 0.5},
    )

    test_selection = ips.configuration_selection.RandomSelection(data=geopt.atoms, n_configurations=20)
    
    train_data = test_selection.excluded_atoms
    test_data = test_selection.atoms

    for cycle in range(6):
        with project.group(name=f"AL_{cycle}") as group:
            # Define the ML model
            model1 = ips.models.Apax(
                data=train_data,
                validation_data=test_data,
                config="config/apax_1.yaml" if cycle < 5 else "config/apax_3.yaml",
            )
            model2 = ips.models.Apax(
                data=train_data,
                validation_data=test_data,
                config="config/apax_2.yaml" if cycle < 5 else "config/apax_4.yaml",
            )

            ensemble_model = ips.models.EnsembleModel(models=[model1, model2])

            md = ips.calculators.ASEMD(
                    data=geopt.atoms,
                    data_id=-1,
                    model=ensemble_model,
                    thermostat=thermostat,
                    checker_list=[uncertainty_check],
                    steps=50000,
                    sampling_rate=1,
                )
            
            train_data_selection = ips.configuration_selection.ThresholdSelection(
                    data=md, n_configurations=10, min_distance=10
                )

            test_data_selection = ips.configuration_selection.RandomSelection(
                data=md,
                n_configurations=5,
                exclude_configurations=train_data_selection.selected_configurations,
            )

            # evaluate the model
            prediction = ips.analysis.Prediction(model=ensemble_model, data=test_data)
            metrics = ips.analysis.PredictionMetrics(data=prediction)
            force_decomposition = ips.analysis.ForceDecomposition(data=prediction)

            train_data += ips.calculators.CP2KSinglePoint(
                    data=train_data_selection,
                    cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
                    cp2k_shell="cp2k_shell.ssmp",
                    wfn_restart_node=cp2k,
                ).atoms

            test_data += ips.calculators.CP2KSinglePoint(
                data=test_data_selection,
                cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
                cp2k_shell="cp2k_shell.ssmp",
                wfn_restart_node=cp2k,
            ).atoms

            md_forces_uncertainty = ips.analysis.ForcesUncertaintyHistogram(data=md.atoms)
            md_energy_uncertainty = ips.analysis.EnergyUncertaintyHistogram(data=md.atoms)

            volume_scan = ips.analysis.BoxScale(data=geopt.atoms, mapping=mapping, model=ensemble_model, start=0.9, data_id=-1)

    with project.group(name="bootstrap_0") as group:
        bootstrap_train_data = ips.bootstrap.RotateMolecules(
            data=geopt.atoms,
            data_id=-1,
            n_configurations=10,
            maximum=10 * 3.1415 / 180,  # deg max rotation
            include_original=False,
            seed=1
        ).atoms + ips.bootstrap.TranslateMolecules(
            data=geopt.atoms,
            data_id=-1,
            n_configurations=10,
            maximum=0.2, # Ang max molecular displacement
            include_original=False,
            seed=1
        ).atoms

        bootstrap_test_data = ips.bootstrap.RotateMolecules(
            data=geopt.atoms,
            data_id=-1,
            n_configurations=5,
            maximum=10 * 3.1415 / 180,  # deg max rotation
            include_original=False,
            seed=2,
            name="RotateMolecules_test"
        ).atoms + ips.bootstrap.TranslateMolecules(
            data=geopt.atoms,
            data_id=-1,
            n_configurations=5,
            maximum=0.2, # Ang max molecular displacement
            include_original=False,
            seed=2,
            name="TranslateMolecules_test"
        ).atoms

        train_data += ips.calculators.CP2KSinglePoint(
                data=bootstrap_train_data,
                cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
                cp2k_shell="cp2k_shell.ssmp",
                wfn_restart_node=cp2k,
            ).atoms

        test_data += ips.calculators.CP2KSinglePoint(
            data=bootstrap_test_data,
            cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
            cp2k_shell="cp2k_shell.ssmp",
            wfn_restart_node=cp2k,
        ).atoms

        model1 = ips.models.Apax(
            data=train_data,
            validation_data=test_data,
            config="config/apax_1.yaml" if cycle < 5 else "config/apax_3.yaml",
        )
        model2 = ips.models.Apax(
            data=train_data,
            validation_data=test_data,
            config="config/apax_2.yaml" if cycle < 5 else "config/apax_4.yaml",
        )

        ensemble_model = ips.models.EnsembleModel(models=[model1, model2])

        md = ips.calculators.ASEMD(
                data=geopt.atoms,
                data_id=-1,
                model=ensemble_model,
                thermostat=thermostat,
                checker_list=[uncertainty_check],
                steps=50000,
                sampling_rate=100,
            )

        # evaluate the model
        prediction = ips.analysis.Prediction(model=ensemble_model, data=test_data)
        metrics = ips.analysis.PredictionMetrics(data=prediction)

        force_decomposition = ips.analysis.ForceDecomposition(data=prediction)
        volume_scan = ips.analysis.BoxScale(data=geopt.atoms, mapping=mapping, model=ensemble_model, start=0.9, data_id=-1)

project.build(nodes=[group]) # 

Running DVC command: 'stage add --name bootstrap_0_RotateMolecules --force ...'
Running DVC command: 'stage add --name bootstrap_0_TranslateMolecules --force ...'
Running DVC command: 'stage add --name bootstrap_0_RotateMolecules_test --force ...'
Running DVC command: 'stage add --name bootstrap_0_TranslateMolecules_test --force ...'
Running DVC command: 'stage add --name bootstrap_0_CP2KSinglePoint_1 --force ...'
Running DVC command: 'stage add --name bootstrap_0_CP2KSinglePoint_2 --force ...'
Running DVC command: 'stage add --name bootstrap_0_MLModel --force ...'
Running DVC command: 'stage add --name bootstrap_0_MLModel_1 --force ...'
Running DVC command: 'stage add --name bootstrap_0_EnsembleModel --force ...'
Running DVC command: 'stage add --name bootstrap_0_ASEMD --force ...'
Running DVC command: 'stage add --name bootstrap_0_ASEMD_checker_list_0 --force ...'
Running DVC command: 'stage add --name bootstrap_0_ASEMD_thermostat --force ...'
Running DVC command: 'stage add --name b